# Creating a dataframe from Toronto postcodes wikipedia page


***
**Cut & pasted install & import commands from the NYC Neighborhood Lab**

Most aren't yet needed - but thought they'd be needed eventually, so included
***

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be UPDATED:

    ca-certificates: 2018.03.07-0      anaconda --> 2018.10.15-ha4d7672_0 conda-forge
    certifi:         2018.10.15-py36_0 anaconda --> 2018.10.15-py36_1000  conda-forge
    conda:           4.5.11-py36_0     anaconda --> 4.5.11-py36_1000      conda-forge
    openssl:         1.0.2p-h14c3975_0 anaconda --> 1.0.2p-h470a237_1     conda-forge

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


***
*Include BeautifulSoup*
***

In [2]:
from bs4 import BeautifulSoup

***
*When I installed the html parsers below - each complained about missing msgpack*

*So I installed msgpack*
***

In [3]:
!conda install -c anaconda msgpack-python --yes
print("msgpack installed")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - msgpack-python


The following packages will be DOWNGRADED:

    ca-certificates: 2018.10.15-ha4d7672_0 conda-forge --> 2018.03.07-0      anaconda
    certifi:         2018.10.15-py36_1000  conda-forge --> 2018.10.15-py36_0 anaconda
    conda:           4.5.11-py36_1000      conda-forge --> 4.5.11-py36_0     anaconda
    openssl:         1.0.2p-h470a237_1     conda-forge --> 1.0.2p-h14c3975_0 anaconda

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
msgpack installed


***
*lxml - recommended html parser in the Beautiful Soup youtube video*
***

In [4]:
!pip install lxml #html parser

***
*Didn't use html5lib - so commented out*
***

In [5]:
#Alternative html parser mentioned in video - didn't use, so commented out
#!pip install html5lib #alternate html parser

***
*Access wikipedia link - 'page' contains BeautifulSoup version of wiki page*
***

In [6]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(wikipedia_link).text
page = BeautifulSoup(source, 'lxml')
#Uncomment line below to see html source page
#print(page.prettify())

***
## Main body of program
tbody contains the html table body

created 3 empty lists to add the postcode, borough, neighorhood as they are determined
the for tr loops through each of the table rows
the for td loops through the 3 items (postcode, borough, neighborhood) on each row

Once the row is parsed,
If the borough is "Not assigned" do nothing
If the neighborhood is "Not assigned" but borough isn't, set neighborhood to borough
If the borough is the same as the previous borough, add neighborhood to the last neighborhood item
Otherwise, add postcode, borough, neighborhood to their respective lists

Once the for loops complete, convert to a dataframe
***


In [7]:
tbody= page.find('tbody')
#print(tbody.prettify())

postcode_list=[]
borough_list=[]
neighborhood_list=[]
postcode=''
borough='Not assigned'
neighborhood='Not assigned'
for tr in tbody.find_all('tr'):
    i=0
    prev_borough=borough
    for td in tr.find_all('td'):
        item= (td.text)
        if i==0:
            postcode= item
        elif i==1:
            borough= item
        else:
            neighborhood= item.strip('\n')
        i+=1
    #print ("i= ", i, "postcode= ", postcode, "borough= ", borough, "neighborhood= ", neighborhood)
    
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood= borough
        if borough != prev_borough:
            postcode_list.append(postcode)
            borough_list.append(borough)
            neighborhood_list.append(neighborhood)
        else:
            neighborhood_list[-1]=neighborhood_list[-1]+", "+neighborhood
    #print(postcode_list)
postcode_df= pd.DataFrame({'Postcode':postcode_list, 'Borough':borough_list, 'Neighborhood':neighborhood_list})
postcode_df.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Victoria Village"
1,M5A,Downtown Toronto,"Harbourfront, Regent Park"
2,M6A,North York,"Lawrence Heights, Lawrence Manor"
3,M7A,Queen's Park,Queen's Park
4,M9A,Etobicoke,Islington Avenue
5,M1B,Scarborough,"Rouge, Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens, Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson, Garden District"
9,M6B,North York,Glencairn


In [8]:
postcode_df.shape

(88, 3)